In [ ]:
import numpy as np
import pandas as pd

In [ ]:
df= pd.read_csv('../input/breast-cancer-wisconsin-data/data.csv') 

In [ ]:
df.isna().sum()
list = ['Unnamed: 32','id']
df = df.drop(list,axis =1)

In [ ]:
df.head()

In [ ]:
import seaborn as sns
ax = sns.countplot(x = 'diagnosis',data = df)

In [ ]:
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
df['diagnosis'] = lb.fit_transform(df['diagnosis'].values)
y = df.diagnosis

In [ ]:
x = df.drop('diagnosis',axis =1)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("ggplot")
f,ax = plt.subplots(figsize=(18, 18))
mask = np.triu(np.ones_like(x.corr(), dtype=np.bool))
sns.heatmap(x.corr(), mask=mask, annot=True, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5},fmt= '.1f',ax=ax)

In [ ]:
list = ['radius_worst','area_worst','perimeter_worst','perimeter_mean','area_mean','area_se','perimeter_se']
x =x.drop(list,axis=1)

In [ ]:
x

In [ ]:
x.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, y,
                                                    test_size=0.20,
                                                    random_state=42,
                                                    stratify = y
                                                    
                                                  )

In [ ]:
from imblearn.over_sampling import SMOTE 
sm = SMOTE(random_state = 2) 
X_train, y_train =sm.fit_sample(X_train, y_train)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [ ]:
X_train.shape

In [ ]:
from keras.layers import Input, Dense, LeakyReLU
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
model = Sequential()
model.add(Dense(512, input_dim=23))
model.add(LeakyReLU(alpha=0.3))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(LeakyReLU(alpha=0.3))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.3))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

In [ ]:
import keras
optimizer  = keras.optimizers.Adam(learning_rate=0.0001,amsgrad=False)

In [ ]:
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=[keras.metrics.Recall(),keras.metrics.Precision()])

In [ ]:
from keras.callbacks import LearningRateScheduler
import math
from keras.callbacks import EarlyStopping 
from keras.callbacks import ModelCheckpoint
es = EarlyStopping(monitor='val_loss', mode='min', patience = 100,restore_best_weights = True)
filepath="weights.hdf5"
cp_callback =ModelCheckpoint(filepath,
                            monitor='val_loss',
                            save_best_only=True,
                            verbose=1)



In [ ]:
history = model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 2,callbacks =[cp_callback,es,lrate])

In [ ]:
history.history['val_recall_19']

In [ ]:
for i in range(45):
    history = model.fit(X_train,y_train,validation_data = (X_test,y_test),epochs = 2,callbacks =[cp_callback,es,lrate])
    if history

In [ ]:
y_pred = model.predict(X_test)
y_pred = (y_pred>0.5)

In [ ]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten().astype(int)})
df

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
X_t = sc.transform(x)
y_total = model.predict(X_t)

In [ ]:
y_total = (y_total>0.5)

In [ ]:
df = pd.DataFrame({'Actual': y, 'Predicted': y_total.flatten().astype(int)})
df

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y, y_total)
sns.heatmap(cm,annot=True)
plt.savefig('h.png')

In [ ]:

from sklearn.metrics import classification_report
print(classification_report(y, y_total))

In [ ]:
plt.plot(history.history['recall_18'])
plt.plot(history.history['val_recall_18'])
plt.title('model recall')
plt.ylabel('recall')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')


In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.show()